# Question Answering

Dobbiamo definire in primis ciò che è il question answering che andremo a trattare.
Il question answering è ESTRATTIVO, ovvero che diamo un contesto e una domanda, tirando fuori dal contesto la risposta. E' diverso dal question answering generico (quello di chatGPT, insomma) che ti produce in output del testo a partire da ciò che ha appreso durante il training. Questo implica che il modello alla base non avrà un'architettura encoder-decoder

## Dataset

Il dataset che useremo è lo Standford Question Answering Dataset (SQuAD)

In [1]:
from datasets import load_dataset

C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_datasets = load_dataset("squad")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [3]:
example = raw_datasets["train"][1]
example

{'id': '5733be284776f4190066117f',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is in front of the Notre Dame Main Building?',
 'answers': {'text': ['a copper statue of Christ'], 'answer_start': [188]}}

In [4]:
example["title"]

'University_of_Notre_Dame'

In [5]:
example["context"]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [6]:
example["answers"]

{'text': ['a copper statue of Christ'], 'answer_start': [188]}

In [7]:
# in train set we don't have any chance for multiple answers or no answers
raw_datasets["train"].filter(lambda x : len(x["answers"]["text"]) !=1)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

In [8]:
raw_datasets["validation"].filter(lambda x : len(x["answers"]["text"]) !=1)[0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


## Tokenizer

La tokenizzazione è un concetto molto interessante all'interno di questo genere di task. Come modello, useremo BERT. Siccome è QA estrattivo, non abbiamo bisogno di un'architettura encoder-decoder e ci facciamo bastare una encoder-only.

Il tokenizer di BERT abbiamo visto che è in grado di prendere due input (caso text entailment). Faremo una cosa simile anche qui, inserendo prima la domanda e poi il context dove è contenuta la risposta. Tuttavia, può accadere che il context sia eccessivamente lungo, così tanto da superare il limite della finestra di contesto del tokenizer di BERT.

Una soluzione valida è quella di spezzare il context in diverse window. Così facendo, passiamo al modello contemporaneamente più input (question+window) e la risposta cadrà in una delle finestre.

C'è un problema: cosa succede se una risposta cade a metà tra le finestre? Inizia a diventare un po' un caos, in quanto il modello avrà pezzi di risposta in input diversi e non sarà in grado di trovarla. Per questo motivo, risolviamo "sovrapponendo" le finestre. E lo faremo in modo che ogni finestra sia sovrapposta a quelle precedenti esattamente a metà. Questo crea ridondanza degli input ma ci assicura di trovare risposta.

In [9]:
from transformers import AutoTokenizer

In [10]:
model_checkpoint = "distilbert-base-cased"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
context = example["context"]
question = example["question"]

In [13]:
inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] What is in front of the Notre Dame Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building \' s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

In [16]:
inputs = tokenizer(
    question,
    context,
    truncation="only_second", # Effettua troncamento solo sul context (second input) e non sulla question
    max_length=100, # Lunghezza massima finestra 
    stride=50, # Quanto c'è di overlapping tra le finestre
    return_overflowing_tokens=True, # Qui diciamo che ci interessano i token della parte  overlappata
    return_offsets_mapping=True # Ci ritorna l'offset che esiste tra le windows e i tokens
)

In [17]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [18]:
# Qui è 0 perchè le finestre appartengono tutte allo stesso
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0]

In [19]:
inputs = tokenizer(
    raw_datasets["train"][:3]["question"],
    raw_datasets["train"][:3]["context"],
    truncation="only_second", # Effettua troncamento solo sul context (second input) e non sulla question
    max_length=100, # Lunghezza massima finestra 
    stride=50, # Quanto c'è di overlapping tra le finestre
    return_overflowing_tokens=True, # Qui diciamo che ci interessano i token della parte  overlappata
    return_offsets_mapping=True # Ci ritorna l'offset che esiste tra le windows e i tokens
)

In [20]:
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]

In [21]:
inputs = tokenizer(
    question,
    context,
    truncation="only_second", # Effettua troncamento solo sul context (second input) e non sulla question
    max_length=100, # Lunghezza massima finestra 
    stride=50, # Quanto c'è di overlapping tra le finestre
    return_overflowing_tokens=True, # Qui diciamo che ci interessano i token della parte  overlappata
    return_offsets_mapping=True # Ci ritorna l'offset che esiste tra le windows e i tokens
)
for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] What is in front of the Notre Dame Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building ' s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the G [SEP]
[CLS] What is in front of the Notre Dame Main Building? [SEP] facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernade [SEP]
[CLS] What is in front of the Notre Dame Main Building? [SEP] of the Sacred Heart. Immediately behind the basilica is the Grotto

In [22]:
# Possiamo notare una cosa: questo output è una lista di liste di tuple
# Ad ogni indice tutto inzia sempre con la domanda, ma poi quando inizia il context il conteggio della posizione si riporta
# esattamente alla posizione contenuta nella frase originale, non alla posizione dell'input.
# P.s: i token speciali non hanno uno spazio, motivo per il quale si indicano con (0,0)
print(inputs["offset_mapping"])

[[(0, 0), (0, 4), (5, 7), (8, 10), (11, 16), (17, 19), (20, 23), (24, 29), (30, 34), (35, 39), (40, 48), (48, 49), (0, 0), (0, 13), (13, 15), (15, 16), (17, 20), (21, 27), (28, 31), (32, 33), (34, 42), (43, 52), (52, 53), (54, 56), (56, 58), (59, 62), (63, 67), (68, 76), (76, 77), (77, 78), (79, 83), (84, 88), (89, 91), (92, 93), (94, 100), (101, 107), (108, 110), (111, 114), (115, 121), (122, 126), (126, 127), (128, 139), (140, 142), (143, 148), (149, 151), (152, 155), (156, 160), (161, 169), (170, 173), (174, 180), (181, 183), (183, 184), (185, 187), (188, 189), (190, 196), (197, 203), (204, 206), (207, 213), (214, 218), (219, 223), (224, 226), (226, 229), (229, 232), (233, 237), (238, 241), (242, 248), (249, 250), (250, 251), (251, 254), (254, 256), (257, 259), (260, 262), (263, 264), (264, 265), (265, 268), (268, 269), (269, 270), (271, 275), (276, 278), (279, 282), (283, 287), (288, 296), (297, 299), (300, 303), (304, 312), (313, 315), (316, 319), (320, 326), (327, 332), (332, 333

### Allineamento dei target

Dobbiamo allineare i target, ora. Questo perchè la posizione dell'answer varia da window a window, adesso. Dobbiamo prendere la posizione assoluta dell'answer e sottrarre l'offset della window

In [62]:
import numpy as np

In [42]:
# Questa funzione (sequence_ids) prende in input un indice (indice della lista esterna) e restituisce gli id.
# Dove troviamo 0, c'è la question. Dove c'è 1, sta la window di contesto. Altrimenti sono token speciali.
# Se usassimo BERT, potremmo avere questi campi direttamente nel "token_type_id"
print(inputs.sequence_ids(0), len(inputs.sequence_ids(0)))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None] 100


In [43]:
# Quel è il problema? Che dividendo la finestra di contesto, la posizione della risposta parte a 181.
# Questo significa che la risposta è presente sia nella finestra 100-200 che in quella 150 2500.
# nel primo caso la lunghezza relativa è 81, nel secondo 31
answer = raw_datasets["train"][1]["answers"]
answer

{'text': ['a copper statue of Christ'], 'answer_start': [188]}

In [44]:
# Il tipo è una lista
type(inputs.sequence_ids(0))

list

In [47]:
# Prima occorrenza di 1
sequence_ids.index(1)

13

In [58]:
# Invertiamo i sequence ids e vediamo dove si trova l'1 ora:
# Siccome l'ultimo token è il SEP, avremo ovviamente che la stringa al contrario è il primo.
# Usiamo questa informazione per tagliare, dalla lunghezza massima, il valore di indice
sequence_ids[::-1].index(1), len(sequence_ids) - sequence_ids[::-1].index(1) -1

(1, 98)

In [61]:
# in alternativa...
np.argwhere(sequence_ids)[0], np.argwhere(sequence_ids)[-1]

(array([13], dtype=int64), array([98], dtype=int64))

In [59]:
# ora vogliamo usare questa funzione in combinazione con gli ids per trovare il context:
sequence_ids = inputs.sequence_ids(0)
ctx_start = sequence_ids.index(1) # -> Ci dice l'indice della prima occorrenza dell'1
ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) -1

ctx_start, ctx_end

(13, 98)

In [69]:
# Ora arriviamo a trovare la posizione della risposta nelle finestre di contesto, a partire dall'indice della risposta nella
# lista. quindi mapperemo la risposta da "char index" a "token index" e poi da "token index (full context) a "window token idx".
# Se non è presente nella finestra, usremo (0,0) come coordinata del token [spazio nullo]
# Faremo questa cosa staticamente per solo la prima finestra. Poi inseriremo tutto in un loop
ans_start_char = answer["answer_start"][0] #188
ans_end_char = ans_start_char + len(answer["text"][0]) #188+ len

offset = inputs["offset_mapping"][0]
start_idx, end_idx = 0, 0
# RICORDA: Offset = tupla (x, y) dove il primo valore (x) è l'indice di partenza del token, y è l'indice di arrivo
# Se il primo valore di offset della prima parola è maggiore dell'indice di inizio della risposta 
# OPPURE se l'ultimo valore di offset dell'ultima parola è minore dell'indice di fine della risposta
if offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char:
    print("(0, 0)") # Fai nulla
else:
    # Troviamo la posizione relativa della rispsota nella finestra di contesto.
    # Loopiamo negli offset, cerchiamo linearmente i token all'interno della tupla
    i = ctx_start # Partiamo da qui per evitare di trovare la posizione nella domanda.
    for start_end_char in offset[ctx_start:]:
        start, end = start_end_char
        if start == ans_start_char:
            start_idx = i
        if end == ans_end_char:
            end_idx = i
            break # il break solo qui
        i+=1

start_idx, end_idx

(53, 57)

In [71]:
# vediamo se è corretto:
input_ids = inputs["input_ids"][0]
input_ids[start_idx:end_idx+1] , tokenizer.decode(input_ids[start_idx:end_idx+1])

([170, 7335, 5921, 1104, 4028], 'a copper statue of Christ')

In [72]:
# Scriviamoci la funzione
def find_answer_token_idx(ctx_start, ctx_end, ans_start_char, ans_end_char, offset):
    start_idx, end_idx = 0, 0
    #Se la risposta non è interamente contenuta nel context:
    if offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char:
        pass
        # Ritorna 0, 0 come settato all'inizio
    else:
        i = ctx_start
        for start_end_char in offset[ctx_start:]:
            start, end = start_end_char
            if start == ans_start_char:
                start_idx = i
            if end == ans_end_char:
                end_idx = i
                break # il break solo qui
            i+=1
    
    return start_idx, end_idx

In [77]:
# Mettiamo tutto in un loop per tutte le finestre di contesto, usando questa funzione!

start_idxs, end_idxs = [], []
for i, offset in enumerate(inputs['offset_mapping']):
    sequence_ids = inputs.sequence_ids(i)
    context_ids = np.argwhere(sequence_ids)
    ctx_start, ctx_end = context_ids[0][0], int(context_ids[-1][0])

    start_idx, end_idx = find_answer_token_idx(ctx_start, ctx_end, ans_start_char, ans_end_char, offset)
    start_idxs.append(start_idx)
    end_idxs.append(end_idx)

start_idxs, end_idxs

([53, 17, 0, 0], [57, 21, 0, 0])

## Tokenizer

A differenza del solito, questa volta i tokenizer saranno due. Uno in fase di training, uno in fase di evaluating.
Key points:
1. Dobbiamo creare le context window
2. Siccome molti dati raggiungeranno la max length, il padding lo inseriamo nel tokenizer e sarà uguale per tutti
3. Dobbiamo allineare le answer alle context window

Il motivo dei due tokenizer è perchè facciamo operazioni diverse sugli output. Nel tokenizer di validazione, infatti, le metriche lavoreranno direttamente sulle stringhe, non sugli IDs

### Train Tokenizer

In [79]:
# In primo luogo, alcune domande sono mal formattate ed hanno degli spazi prima e dopo. Vediamo quali:
for q in raw_datasets["train"]["question"][:1000]:
    if q.strip() != q:
        print(q)

In what city and state did Beyonce  grow up? 
 The album, Dangerously in Love  achieved what spot on the Billboard Top 100 chart?
Which song did Beyonce sing at the first couple's inaugural ball? 
What event did Beyoncé perform at one month after Obama's inauguration? 
Where was the album released? 
What movie influenced Beyonce towards empowerment themes? 


In [84]:
 # Questi valori vengono da esperimenti di google:
max_lenght = 384
stride = 128

def tokenizer_fn_train(batch):

    questions = [q.strip() for q in batch["question"]]

    inputs = tokenizer(
        questions,
        batch["context"],
        max_length=max_lenght,
        stride=stride,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Siccome non avremo bisogno delle colonne dopo, le rimuoviamo per non ritrovarcele in uscita
    offset_mapping = inputs.pop("offset_mapping")
    orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
    answers = batch["answers"]

    start_idxs, end_idxs = [], []

    for i, offset in enumerate(offset_mapping):
        # Qui noi stiamo semplicemente recuperando la risposta dal dataset originale (raw) perchè il tokenizer crea quello espanso
        sample_idx = orig_sample_idxs[i]
        answer = answers[sample_idx]
        
        ans_start_char = answer["answer_start"][0]
        ans_end_char = ans_start_char + len(answer["text"][0])
        
        sequence_ids = inputs.sequence_ids(i)
        context_ids = np.argwhere(sequence_ids)
        ctx_start, ctx_end = context_ids[0][0], int(context_ids[-1][0])

        start_idx, end_idx = find_answer_token_idx(ctx_start, ctx_end, ans_start_char, ans_end_char, offset)
        start_idxs.append(start_idx)
        end_idxs.append(end_idx)

    inputs["start_positions"] = start_idxs
    inputs["end_positions"] = end_idxs
    return inputs    

In [85]:
train_dataset = raw_datasets["train"].map(tokenizer_fn_train, batched=True, remove_columns=raw_datasets["train"].column_names)

Map: 100%|██████████████████████████████████████████████████████████████| 87599/87599 [00:24<00:00, 3566.96 examples/s]


In [98]:
len(raw_datasets["train"]), len(train_dataset)

(87599, 88729)

### Eval Tokenizer

Il tokenizer della validazione non necessita dei target. La risposta sarò comparata direttamente sulle stringhe

Una seconda differenza riguarda l'offset mapping: metteremo a "None" i token relativi alla domanda, scopriremo perchè dopo

In [87]:
# Terremo gli "id", gli identificatori univoci del campione nel dato. Serviranno per matchare correttamente in fase di eval
raw_datasets["validation"][0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


In [88]:
def tokenizer_fn_validation(batch):

    questions = [q.strip() for q in batch["question"]]

    inputs = tokenizer(
        questions,
        batch["context"],
        max_length=max_lenght,
        stride=stride,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Siccome non avremo bisogno delle colonne dopo, le rimuoviamo per non ritrovarcele in uscita
    orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
    sample_ids = [] # Qui ci saranno gli id alfanumerici

    for i in range(len(inputs["input_ids"])):
        sample_idx = orig_sample_idxs[i]
        sample_ids.append(batch["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]

        # Lascia l'offset invariato se è relativo al contesto, altrimenti diventa un None
        inputs["offset_mapping"][i] = [x if sequence_ids[j] == 1 else None for j, x  in enumerate(offset)]

    inputs["sample_id"] = sample_ids

    return inputs    

In [94]:
validation_dataset = raw_datasets["validation"].map(
    tokenizer_fn_validation, batched=True, remove_columns=raw_datasets["validation"].column_names
)

Map: 100%|██████████████████████████████████████████████████████████████| 10570/10570 [00:03<00:00, 2931.67 examples/s]


In [99]:
len(raw_datasets["validation"]), len(validation_dataset)

(10570, 10822)